In [90]:
import os
from PIL import Image
import numpy as np
from scipy.spatial import distance
import math
import tqdm

from scipy import spatial
import cv2
from skimage import feature

# PIXEL BY PIXEL
<hr>

In [91]:
#read all images
def loadImages(path = 'awe/'):
    subfolders = [ f.path + "/" for f in os.scandir(path) if f.is_dir() ]
    return [ f.path for images in subfolders for f in os.scandir(images) if f.name[-3:] == 'png']

#convert image to greyscale and resize it to 128,128 if the other size not is given as argument
def convert(image,size=(128, 128)):
    return Image.open(image).convert("L").resize(size)

In [92]:
#save all path images in one array
imagePaths = loadImages()

In [93]:
#get image as one vector
def pixelByPixel(image,size=(128,128)):
    im = convert(image,size)
    return np.array(list(im.getdata()))

In [94]:
#calculate distances between 2 vectors
def euclidian(im1,im2):
    return distance.euclidean(im1, im2)

In [95]:
#calculate distances between 2 vectors
def cosine(im1,im2):
    return distance.cosine(im1, im2)

In [7]:
#create list of all images saved as pixel by pixel
def load(size=(128,128)):
    images = []
    for i in loadImages():
        images.append(pixelByPixel(i,size)) 
    return images

In [8]:
#create list of all images saved as pixel by pixel 128x128
images = load()

In [9]:
#create list of all images saved as pixel by pixel 256x256
imagesSize = load((64,64))

In [10]:
#accuracy EUCLID:calculate distance between each vector and save for each vector minimum distance
def euclidDist(images):
    minDist = []
    for i in tqdm.tqdm(range(len(images))):
        arr = []
        for j in range(len(images)):
            if i!=j:
                arr.append((i,j,euclidian(images[i],images[j])))
        minDist.append(sorted(arr, key=lambda x: x[2])[0])
    return sum([imagePaths[minDist[i][0]].split("/")[-2] == imagePaths[minDist[i][1]].split("/")[-2] for i in range(len(minDist))])/len(minDist)

In [11]:
#pixel by pixel euclid distance 128x128
euclidDist(images)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:57<00:00, 17.33it/s]


0.132

In [12]:
#pixel by pixel euclid distance 64x64
euclidDist(imagesSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:31<00:00, 32.19it/s]


0.132

In [13]:
#accuracy COSINE:calculate distance between each vector and save for each vector minimum distance
def cosineDist(images):
    minDistcosin = []
    for i in tqdm.tqdm(range(len(images))):
        arr = []
        for j in range(len(images)):
            if i!=j:
                arr.append((i,j,cosine(images[i],images[j])))
        minDistcosin.append(sorted(arr, key=lambda x: x[2])[0])
    return sum([imagePaths[minDistcosin[i][0]].split("/")[-2] == imagePaths[minDistcosin[i][1]].split("/")[-2] for i in range(len(minDistcosin))])/len(minDistcosin)

In [14]:
#pixel by pixel cosine distance 128x128
cosineDist(images)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:47<00:00,  5.99it/s]


0.167

In [15]:
#pixel by pixel cosine distance 64x64
cosineDist(imagesSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:28<00:00, 11.25it/s]


0.168

# Local Binary Pattern
<hr>

In [16]:
#get all pictures as greyscale and size default 128x128
def loadlbp(size=(128,128)):
    imageLBP = []
    for path in imagePaths:
        imageLBP.append(convert(path,size))
    return imageLBP

In [17]:
#get all pictures as greyscale and size default 128x128
imageLBP = loadlbp()

In [18]:
#get all pictures as greyscale and size default 64x64
imageLBPSize = loadlbp((64,64))

In [19]:
# get matrix
def matrix(image,currentPos,size=1):
    region = []
    for y in range(currentPos[1]-size,currentPos[1]+size+1):
        if y < 0 or y >= image.size[1]:
            row = [0 for _ in range(size * 2 + 1)]
        else:
            row = []
            for x in range(currentPos[0]-size,currentPos[0]+size+1):
                if x < 0 or x >= image.size[0]:
                    row.append(0)
                else:
                    row.append(image.getpixel((x,y)))
        region.append(row)
    return region        

In [20]:
# imageLBP[0]

In [21]:
# matrix(imageLBP[0],(0,0),2)

In [22]:
# matrix(imageLBP[0],(0,0),3)

In [23]:
#change values of matrix to 0 and 1
def lbp(matrix):
    binaryarr = []
    centery,centerx = int(len(matrix) /2),int(len(matrix[0]) /2)
    for j in range(len(matrix)):
        row = []
        for i in range(len(matrix[j])):
            if (i,j) != (centerx,centery):
                row.append(int(matrix[centery][centerx] < matrix[j][i]))
            else:
                row.append(0)
        binaryarr.append(row)
        
 
    return binaryarr

In [24]:
# lbp(matrix(imageLBP[0],(0,0),))

In [25]:
#binary matrix transform to array
def matrixToArray(matrix):
    arr = np.array(matrix)
    return list(arr[0,:]) + list(arr[1:, -1]) + list(arr[-1, -2::-1]) + list(reversed(arr[1:-1, 0]))

In [26]:
#binary array to decimal values
def binToDec(binary):
    val = 0
    for i in range(len(binary)):
        val = val * 2 + binary[i]
    return val 

In [27]:
#all images to lbp look
def imageToLBP(imageLBP,size=1):
    newImages = []
    for i in tqdm.tqdm(range(len(imageLBP))):
        newImageLBP = imageLBP[i].copy()
        for y in range(imageLBP[i].size[1]):
            for x in range(imageLBP[i].size[0]):
                neighbour = matrix(imageLBP[i],(x,y),size)
                newImageLBP.putpixel((x, y), int(binToDec(matrixToArray(lbp(neighbour)))))
        newImages.append(newImageLBP)
    return newImages

In [28]:
#all images to lbp look
newImages = imageToLBP(imageLBP)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:50<00:00,  1.69it/s]


In [29]:
#all images to lbp look different range = 2
newImages2 = imageToLBP(imageLBP,2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [21:52<00:00,  1.31s/it]


In [30]:
#all images to lbp look 64x64
newImageSize = imageToLBP(imageLBPSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:37<00:00,  3.60it/s]


In [31]:
#all images to lbp look 64x64  different range 3
newImageSize2 = imageToLBP(imageLBPSize,2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:21<00:00,  2.62it/s]


#### WITHOUT HISTOGRAM
<hr>

In [32]:
#create list of all images saved as pixel by pixel
def lbppixelbypixel(newImages):
    images = []
    for im in newImages:
        images.append(np.array(list(im.getdata()))) 
    return images

In [33]:
images1 = lbppixelbypixel(newImages)

In [34]:
images2 = lbppixelbypixel(newImages2)

In [35]:
#create list of all images saved as pixel by pixel 64x64
imagesSize1 = lbppixelbypixel(newImageSize)

In [36]:
#create list of all images saved as pixel by pixel 64x64
imagesSize2 = lbppixelbypixel(newImageSize2)

In [37]:
#accuracy EUCLID:calculate distance between each vector and save for each vector minimum distance WITHOUT HISTOGRAM
euclidDist(images1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:13<00:00, 13.63it/s]


0.208

In [38]:
#accuracy EUCLID:calculate distance between each vector and save for each vector minimum distance WITHOUT HISTOGRAM range 2
euclidDist(images2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:10<00:00, 14.16it/s]


0.027

In [39]:
#accuracy with euclid 64x64
euclidDist(imagesSize1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:30<00:00, 32.62it/s]


0.277

In [40]:
#accuracy with euclid 64x64 range 2
euclidDist(imagesSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:34<00:00, 29.03it/s]


0.048

In [41]:
#accuracy COSINE:calculate distance between each vector and save for each vector minimum distance WITHOUT HISTOGRAM
cosineDist(images1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:56<00:00,  5.67it/s]


0.229

In [42]:
#accuracy COSINE:calculate distance between each vector and save for each vector minimum distance WITHOUT HISTOGRAM range 2
cosineDist(images2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:07<00:00,  5.34it/s]


0.023

In [43]:
#accuracy COSINE:calculate distance between each vector and save for each vector minimum distance WITHOUT HISTOGRAM 64x64
cosineDist(imagesSize1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:31<00:00, 10.95it/s]


0.283

In [44]:
#accuracy COSINE:calculate distance between each vector and save for each vector minimum distance WITHOUT HISTOGRAM 64x64 range 2
cosineDist(imagesSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:31<00:00, 10.98it/s]


0.04

#### WITH HISTOGRAM
<hr>

In [45]:
#create dict of 256 fields of 0 and sum up repetition of each number
#save dictionary for each image in qwerty 
def dictionary(newImages):
    qwerty = []
    for i in tqdm.tqdm(range(len(newImages))):
        proba = newImages[i].copy()
        values = {i:0 for i in range(256)}
        for y in range(proba.size[1]):
            for x in range(proba.size[0]):
                values[proba.getpixel((x,y))]+=1
        qwerty.append(list(values.values()))
    return qwerty

In [46]:
#dictionary of values 128x128
qwerty = dictionary(newImages)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:18<00:00, 52.78it/s]


In [47]:
#dictionary of values 128x128 rang e2
qwerty2 = dictionary(newImages2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 50.46it/s]


In [48]:
#dictionary of values 64x64
qwertySize = dictionary(newImageSize)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 212.09it/s]


In [49]:
#dictionary of values 64x64 range 2
qwertySize2 = dictionary(newImageSize2)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 203.38it/s]


In [50]:
# accuracy EUCLIDIAN: distance between HISTOGRAMS 
def euclidDistHist(qwerty,images1):
    minDist1 = []
    for i in tqdm.tqdm(range(len(qwerty))):
        arr = []
        for j in range(len(qwerty)):
            if i!=j:
                arr.append((i,j,euclidian(qwerty[i],qwerty[j])+euclidian(images1[i],images1[j])))
        minDist1.append(sorted(arr, key=lambda x: x[2])[0])
        
    return sum([imagePaths[minDist1[i][0]].split("/")[-2] == imagePaths[minDist1[i][1]].split("/")[-2] for i in range(len(minDist1))])/len(minDist1)

In [51]:
euclidDistHist(qwerty,images1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:19<00:00,  7.18it/s]


0.248

In [52]:
euclidDistHist(qwerty2,images2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:15<00:00,  7.37it/s]


0.035

In [53]:
euclidDistHist(qwertySize,imagesSize1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:52<00:00,  8.89it/s]


0.306

In [54]:
euclidDistHist(qwertySize2,imagesSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:45<00:00,  9.46it/s]


0.073

In [55]:
# accuracy COSINE: distance between HISTOGRAMS 
def cosineDistHist(qwerty,images1):
    minDist1 = []
    for i in tqdm.tqdm(range(len(qwerty))):
        arr = []
        for j in range(len(qwerty)):
            if i!=j:
                arr.append((i,j,cosine(qwerty[i],qwerty[j])+cosine(images1[i],images1[j])))
        minDist1.append(sorted(arr, key=lambda x: x[2])[0])
        
    return sum([imagePaths[minDist1[i][0]].split("/")[-2] == imagePaths[minDist1[i][1]].split("/")[-2] for i in range(len(minDist1))])/len(minDist1)

In [56]:
cosineDistHist(qwerty,images1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:48<00:00,  2.87it/s]


0.228

In [57]:
cosineDistHist(qwerty2,images2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:14<00:00,  3.18it/s]


0.024

In [58]:
cosineDistHist(qwertySize,imagesSize1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:28<00:00,  4.79it/s]


0.262

In [59]:
cosineDistHist(qwertySize2,imagesSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:15<00:00,  5.11it/s]


0.041

# openCV
<hr>


In [60]:
def opencv(path,size=(128,128)):
    image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    return cv2.resize(image,size,cv2.INTER_LINEAR)

In [61]:
def describe(image,method,p=8,r=1):
    lbp = feature.local_binary_pattern(image, p, r, method=method)
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 257), range=(0, 256))
   
    return lbp,hist

In [62]:
def insertmetod(method,size=(128,128),p=8,r=1):
    openlbp = []
    openhist = []
    for i in range(len(imagePaths)):
        newOpen = opencv(imagePaths[i],size)
        lbp,hist = describe(newOpen,method,p,r)
        openlbp.append(lbp)
        openhist.append(hist)
    return openlbp,openhist

In [63]:
opendefault, opendefaulthist = insertmetod('default')

In [64]:
opendefault2, opendefaulthist2 = insertmetod('default',(128,128),16,2)

In [65]:
openuniform,openuniformhist = insertmetod('uniform')

In [66]:
openuniform2,openuniformhist2 = insertmetod('uniform',(128,128),16,2)

In [67]:
openror, openrorhist = insertmetod('ror')

In [68]:
openror2, openrorhist2 = insertmetod('ror',(128,128),16,2)

In [69]:
opendefaultSize, opendefaulthistSize = insertmetod('default',(64,64))

In [70]:
opendefaultSize2, opendefaulthistSize2 = insertmetod('default',(64,64),16,2)

In [71]:
openuniformSize,openuniformhistSize = insertmetod('uniform',(64,64),16,2)

In [72]:
openuniformSize2,openuniformhistSize2 = insertmetod('uniform',(64,64),16,2)

In [73]:
openrorSize, openrorhistSize = insertmetod('ror',(64,64))

In [74]:
openrorSize2, openrorhistSize2 = insertmetod('ror',(64,64),16,2)

In [75]:
#EUCLID distance no hist
def distance(imagePaths,openlbp):
    minDistopen = []
    for i in tqdm.tqdm(range(len(openlbp))):
        arr = []
        for j in range(len(openlbp)):
            if i!=j:
                arr.append((i,j,np.linalg.norm(openlbp[i]-openlbp[j])))
        minDistopen.append(sorted(arr, key=lambda x: x[2])[0])
    #accuracy with euclid
    return sum([imagePaths[minDistopen[i][0]].split("/")[-2] == imagePaths[minDistopen[i][1]].split("/")[-2] for i in range(len(minDistopen))])/len(minDistopen)

In [76]:
distance(imagePaths,opendefault)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:34<00:00, 29.37it/s]


0.223

In [77]:
distance(imagePaths,opendefault2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:35<00:00, 27.87it/s]


0.277

In [78]:
distance(imagePaths,openuniform)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:41<00:00, 24.16it/s]


0.014

In [79]:
distance(imagePaths,openuniform2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:42<00:00, 23.56it/s]


0.014

In [80]:
distance(imagePaths,openror)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:39<00:00, 25.12it/s]


0.014

In [81]:
distance(imagePaths,openror2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:39<00:00, 25.59it/s]


0.007

In [82]:
distance(imagePaths,opendefaultSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:26<00:00, 37.65it/s]


0.243

In [83]:
distance(imagePaths,opendefaultSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.32it/s]


0.31

In [84]:
distance(imagePaths,openuniformSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:30<00:00, 32.77it/s]


0.033

In [85]:
distance(imagePaths,openuniformSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:28<00:00, 35.40it/s]


0.033

In [86]:
distance(imagePaths,openrorSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:30<00:00, 32.52it/s]


0.015

In [87]:
distance(imagePaths,openrorSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:29<00:00, 33.78it/s]


0.031

In [96]:
#COSINE distance no hist
def distanceCosine(imagePaths,openlbp):
    minDistopen = []
    for i in tqdm.tqdm(range(len(openlbp))):
        arr = []
        for j in range(len(openlbp)):
            if i!=j:
                arr.append((i,j,cosine(np.array(openlbp[i]).ravel(),np.array(openlbp[j]).ravel())))
        minDistopen.append(sorted(arr, key=lambda x: x[2])[0])
    #accuracy with euclid
    return sum([imagePaths[minDistopen[i][0]].split("/")[-2] == imagePaths[minDistopen[i][1]].split("/")[-2] for i in range(len(minDistopen))])/len(minDistopen)

In [97]:
distanceCosine(imagePaths,opendefault)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:39<00:00,  6.25it/s]


0.114

In [98]:
distanceCosine(imagePaths,opendefault2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:49<00:00,  5.91it/s]


0.187

In [99]:
distanceCosine(imagePaths,openuniform)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:48<00:00,  5.94it/s]


0.012

In [100]:
distanceCosine(imagePaths,openuniform2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:34<00:00,  6.46it/s]


0.013

In [101]:
distanceCosine(imagePaths,openror)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:43<00:00,  6.11it/s]


0.01

In [102]:
distanceCosine(imagePaths,openror2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:34<00:00,  6.46it/s]


0.013

In [103]:
distanceCosine(imagePaths,opendefaultSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:39<00:00, 10.04it/s]


0.166

In [104]:
distanceCosine(imagePaths,opendefaultSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:30<00:00, 11.00it/s]


0.232

In [105]:
distanceCosine(imagePaths,openuniformSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:27<00:00, 11.37it/s]


0.046

In [106]:
distanceCosine(imagePaths,openuniformSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:20<00:00, 12.37it/s]


0.046

In [107]:
distanceCosine(imagePaths,openrorSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:20<00:00, 12.37it/s]


0.043

In [108]:
distanceCosine(imagePaths,openrorSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:35<00:00, 10.50it/s]


0.101

In [109]:
def distancehist(imagePaths,openlbp,openhist):
    #EUCLIDIAN: distance between OPENCV DEFUALT HISTOGRAM
    minDistopen1 = []
    for i in tqdm.tqdm(range(len(openlbp))):
        arr = []
        for j in range(len(openlbp)):
            if i!=j:
                arr.append((i,j,np.linalg.norm(openlbp[i]-openlbp[j])+np.linalg.norm(openhist[i]-openhist[j])))
        minDistopen1.append(sorted(arr, key=lambda x: x[2])[0])
    #accuracy with euclid
    return sum([imagePaths[minDistopen1[i][0]].split("/")[-2] == imagePaths[minDistopen1[i][1]].split("/")[-2] for i in range(len(minDistopen1))])/len(minDistopen1)

In [110]:
distancehist(imagePaths,opendefault,opendefaulthist)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:53<00:00, 18.81it/s]


0.261

In [111]:
distancehist(imagePaths,opendefault2,opendefaulthist2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:52<00:00, 19.06it/s]


0.277

In [112]:
distancehist(imagePaths,openuniform, openuniformhist)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:55<00:00, 18.01it/s]


0.016

In [113]:
distancehist(imagePaths,openuniform2, openuniformhist2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:14<00:00, 13.36it/s]


0.016

In [114]:
distancehist(imagePaths,openror,openrorhist)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:10<00:00, 14.14it/s]


0.031

In [115]:
distancehist(imagePaths,openror2,openrorhist2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:54<00:00, 18.32it/s]


0.007

In [116]:
distancehist(imagePaths,opendefaultSize,opendefaulthistSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:37<00:00, 26.69it/s]


0.293

In [117]:
distancehist(imagePaths,opendefaultSize2,opendefaulthistSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:36<00:00, 27.34it/s]


0.31

In [118]:
distancehist(imagePaths,openuniformSize, openuniformhistSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:33<00:00, 29.43it/s]


0.039

In [119]:
distancehist(imagePaths,openuniformSize2, openuniformhistSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:39<00:00, 25.45it/s]


0.039

In [120]:
distancehist(imagePaths,openrorSize,openrorhistSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:37<00:00, 26.49it/s]


0.053

In [121]:
distancehist(imagePaths,openrorSize2,openrorhistSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:36<00:00, 27.60it/s]


0.032

In [123]:
def distancehistCosine(imagePaths,openlbp,openhist):
    #COSINE: distance between OPENCV DEFUALT HISTOGRAM
    minDistopen1 = []
    for i in tqdm.tqdm(range(len(openlbp))):
        arr = []
        for j in range(len(openlbp)):
            if i!=j:
#                 arr.append((i,j,cosine(openlbp[i]-openlbp[j])+ cosine(openhist[i]-openhist[j])))
                arr.append((i,j,cosine(np.array(openlbp[i]).ravel(),np.array(openlbp[j]).ravel()) + cosine(np.array(openhist[i]).ravel(),np.array(openhist[j]).ravel())))
        minDistopen1.append(sorted(arr, key=lambda x: x[2])[0])
    #accuracy with euclid
    return sum([imagePaths[minDistopen1[i][0]].split("/")[-2] == imagePaths[minDistopen1[i][1]].split("/")[-2] for i in range(len(minDistopen1))])/len(minDistopen1)

In [124]:
distancehistCosine(imagePaths,opendefault,opendefaulthist)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:25<00:00,  4.86it/s]


0.182

In [125]:
distancehistCosine(imagePaths,opendefault2,opendefaulthist2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:48<00:00,  4.38it/s]


0.16

In [126]:
distancehistCosine(imagePaths,openuniform, openuniformhist)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:15<00:00,  3.91it/s]


0.048

In [127]:
distancehistCosine(imagePaths,openuniform2, openuniformhist2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:49<00:00,  4.36it/s]


0.051

In [128]:
distancehistCosine(imagePaths,openror,openrorhist)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:40<00:00,  4.54it/s]


0.051

In [129]:
distancehistCosine(imagePaths,openror2,openrorhist2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:50<00:00,  4.35it/s]


0.021

In [130]:
distancehistCosine(imagePaths,opendefaultSize,opendefaulthistSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:32<00:00,  6.58it/s]


0.215

In [131]:
distancehistCosine(imagePaths,opendefaultSize2,opendefaulthistSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:06<00:00,  5.35it/s]


0.203

In [132]:
distancehistCosine(imagePaths,openuniformSize, openuniformhistSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:02<00:00,  5.48it/s]


0.116

In [133]:
distancehistCosine(imagePaths,openuniformSize2, openuniformhistSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:31<00:00,  6.60it/s]


0.116

In [134]:
distancehistCosine(imagePaths,openrorSize,openrorhistSize)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:23<00:00,  6.96it/s]


0.084

In [135]:
distancehistCosine(imagePaths,openrorSize2,openrorhistSize2)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:22<00:00,  7.00it/s]


0.103